In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
#The url sends a request for 538's presidential polls download and writes it to a csv file
url = 'https://projects.fivethirtyeight.com/polls-page/president_polls.csv'
link = requests.get(url, stream=True)
file = link.content.decode('utf-8')
writer = open('president_polls.csv', 'w')
writer.write(file)
writer.close()

In [3]:
#Read in said file
pres_polls = pd.read_csv('president_polls.csv')

In [4]:
#Convert date columns to datetime
pres_polls['start_date_dt'] = pd.to_datetime(pres_polls.start_date)
pres_polls['end_date_dt'] = pd.to_datetime(pres_polls.end_date)

In [5]:
#Create dataframes for each swing state
penn_df = pres_polls[pres_polls['state'] == 'Pennsylvania']
mich_df = pres_polls[pres_polls['state'] == 'Michigan']
wis_df = pres_polls[pres_polls['state'] == 'Wisconsin']
flo_df = pres_polls[pres_polls['state'] == 'Florida']
ohio_df = pres_polls[pres_polls['state'] == 'Ohio']
ari_df = pres_polls[pres_polls['state'] == 'Arizona']
geo_df = pres_polls[pres_polls['state'] == 'Georgia']
nc_df = pres_polls[pres_polls['state'] == 'North Carolina']
toss_up = [penn_df, mich_df, wis_df, flo_df, ohio_df, ari_df, geo_df, nc_df]

In [6]:
#Set parameters with electoral college points to Trump and Biden along with each state electoral values
biden_electorial_college = 233
trump_electorial_college = 170
electoral_college = {'Michigan': 16, 'Wisconsin': 10, 'Pennsylvania': 20, 'Ohio': 18, 'Florida': 29,
'Arizona': 11, 'Georgia': 16, 'North Carolina': 15}

In [8]:
def simulate_2020(number):
    #For each state we take the current date along with the mean of each Candidates polls for the state
    #Then the remain points are allocated and the candidate with the most points is declared the winner
    #If there is a tie a coin flip is used to determine a winner
    electoral_college = {'Michigan': 10, 'Wisconsin': 16, 'Pennsylvania': 20, 'Ohio': 18, 'Florida': 29,
    'Arizona': 11, 'Georgia': 16, 'North Carolina': 15}
    counter = 0
    biden_wins = 0
    trump_wins = 0
    while counter < number:
        biden_electorial_college = 233
        trump_electorial_college = 170
        for x in toss_up:
            state = x.state.unique()[0]
            biden_pct = x[(x['end_date_dt'] == max(x.end_date_dt)) & (x['candidate_name'] == 'Joseph R. Biden Jr.')].groupby(by='end_date_dt').pct.mean().reset_index()
            trump_pct = x[(x['end_date_dt'] == max(x.end_date_dt)) & (x['candidate_name'] == 'Donald Trump')].groupby(by='end_date_dt').pct.mean().reset_index()
            Biden_points = biden_pct.pct.values[0]
            Trump_points = trump_pct.pct.values[0]
            Difference = 100 - (Biden_points + Trump_points)
            allocate = random.choice(range(0, int(Difference)))
            reallocate = Difference - allocate
            biden_final_points = Biden_points + allocate
            trump_final_points = Trump_points + reallocate
            if biden_final_points > trump_final_points:
                biden_electorial_college += electoral_college[state]
            elif biden_final_points < trump_final_points:
                trump_electorial_college += electoral_college[state]
            else:
                coin_flip = random.choice([1, 2])
                if coin_flip == 1:
                    biden_electorial_college += electoral_college[state]
                elif coin_flip == 2:
                    trump_electorial_college += electoral_college[state]
                else:
                    print('What the hell man?')
        if biden_electorial_college > trump_electorial_college:
            biden_wins += 1
            counter += 1
        elif biden_electorial_college < trump_electorial_college:
            trump_wins += 1
            counter += 1
        else:
            print('Well we\'re really screwed now.')
            counter += 1
    print('In this simulation, Biden won', biden_wins, 'times and Trump won', trump_wins, 'times.')

In [9]:
#Simulate the results
simulate_2020(100)

In this simulation, Biden won 100 times and Trump won 0 times.


In [10]:
#Test with the 2016 polls from 538
df_2016 = pd.read_csv('president_general_polls_2016.csv')

In [11]:
#Create datetime columns and state dataframes
df_2016['end_date_dt'] = pd.to_datetime(df_2016['enddate'])
df_2016 = df_2016[(df_2016['end_date_dt'] < '2016-10-18') & (df_2016['end_date_dt'] > '2016-10-01')]
penn_df = df_2016[df_2016['state'] == 'Pennsylvania']
mich_df = df_2016[df_2016['state'] == 'Michigan']
wis_df = df_2016[df_2016['state'] == 'Wisconsin']
flo_df = df_2016[df_2016['state'] == 'Florida']
ohio_df = df_2016[df_2016['state'] == 'Ohio']
toss_up = [penn_df, mich_df, wis_df, flo_df, ohio_df]

In [12]:
def simulate_2016(number):
    electoral_college = {'Michigan': 16, 'Wisconsin': 10, 'Pennsylvania': 20, 'Ohio': 18, 'Florida': 29,
    'Arizona': 11, 'Georgia': 16, 'North Carolina': 15}
    counter = 0
    clinton_wins = 0
    trump_wins = 0
    while counter < number:
        clinton_electorial_college = 232
        trump_electorial_college = 213
        for x in toss_up:
            state = x.state.unique()[0]
            clinton_pct = x[(x['end_date_dt'] == max(x.end_date_dt))].groupby(by='end_date_dt').rawpoll_clinton.mean().reset_index()
            trump_pct = x[(x['end_date_dt'] == max(x.end_date_dt))].groupby(by='end_date_dt').rawpoll_trump.mean().reset_index()
            clinton_points = clinton_pct.rawpoll_clinton.values[0]
            Trump_points = trump_pct.rawpoll_trump.values[0]
            Difference = 100 - (clinton_points + Trump_points)
            allocate = random.choice(range(0, int(Difference)))
            reallocate = Difference - allocate
            clinton_final_points = clinton_points + allocate
            trump_final_points = Trump_points + reallocate
            if clinton_final_points > trump_final_points:
                clinton_electorial_college += electoral_college[state]
            elif clinton_final_points < trump_final_points:
                trump_electorial_college += electoral_college[state]
            else:
                coin_flip = random.choice([1, 2])
                if coin_flip == 1:
                    clinton_electorial_college += electoral_college[state]
                elif coin_flip == 2:
                    trump_electorial_college += electoral_college[state]
                else:
                    print('What the hell man?')
        if clinton_electorial_college > trump_electorial_college:
            clinton_wins += 1
            counter += 1
        elif clinton_electorial_college < trump_electorial_college:
            trump_wins += 1
            counter += 1
        else:
            print('Well we\'re really screwed now.')
            counter += 1
    print('In this simulation Clinton won', clinton_wins, 'times and Trump won', trump_wins, 'times.')

In [14]:
#Simulate the 2016
simulate_2016(100)

In this simulation Clinton won 89 times and Trump won 11 times.
